In [23]:
import pandas as pd
import numpy as np
import guidedlda as glda

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

import string

## Reading and Preprocessing data

In [5]:
df1=pd.read_csv("./Data/train_data.csv")
df1 = df1.set_index("Index")
df1.head()

/var/folders/rn/k7vf4gp97fb11dzjbcr7w8bc0000gn/T/ipykernel_76613/686763101.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df1=pd.read_csv("./Data/train_data.csv")


,Venue Index,Name,Date,Rating,Rating Sent,Text,Aspect,Accessibility,Term1,Term2,Term3
Index,,,,,,,,,,,
18594,10,coffeeshop la grotte,2019,5 stars,positive,2nd year coming here and can t say enough good...,0,0,NaN,NaN,NaN
373579,451,pata negra,2019,5 stars,positive,excellent,0,0,NaN,NaN,NaN
128693,112,bagels & beans ijdock,2021,5 stars,positive,"location, very nice and welcoming, with a spl...",0,0,NaN,NaN,NaN
120681,17,plum,2020,5 stars,positive,went to this place on saturday. i had the gril...,0,0,NaN,NaN,NaN
162003,504,caf‚àö¬© thijssen,2021,5 stars,positive,was wer very tasty,0,0,NaN,NaN,NaN


In [8]:
stopwords_list = stopwords.words('english')
punctuations = list(set(string.punctuation))
unwanted_list=punctuations+stopwords_list

In [13]:
def clean_strings(text):
    
    text = str(text)
    text = text.replace("[^a-zA-Z#]", " ")
    
    return text
    
def tokenize(text):
    text = ' '.join([x.lower() for x in word_tokenize(text) if x.lower() not in unwanted_list and len(x)>1])
    text = ' '.join([x.lower() for x in word_tokenize(text) if nltk.pos_tag([x])[0][1].startswith("NN") or nltk.pos_tag([x])[0][1].startswith("JJ")])
    
    return text.strip()

In [14]:
%%time
df1["clean_text"]=df1['Text'].apply(lambda text:clean_strings(str(text)))
df1.head()

CPU times: user 230 ms, sys: 13.7 ms, total: 244 ms
Wall time: 254 ms


,Venue Index,Name,Date,Rating,Rating Sent,Text,Aspect,Accessibility,Term1,Term2,Term3,clean_text
Index,,,,,,,,,,,,
18594,10,coffeeshop la grotte,2019,5 stars,positive,nd year coming here and can t say enough good...,0,0,NaN,NaN,NaN,nd year coming here and can t say enough good...
373579,451,pata negra,2019,5 stars,positive,excellent,0,0,NaN,NaN,NaN,excellent
128693,112,bagels & beans ijdock,2021,5 stars,positive,location very nice and welcoming with a spl...,0,0,NaN,NaN,NaN,location very nice and welcoming with a spl...
120681,17,plum,2020,5 stars,positive,went to this place on saturday i had the gril...,0,0,NaN,NaN,NaN,went to this place on saturday i had the gril...
162003,504,caf‚àö¬© thijssen,2021,5 stars,positive,was wer very tasty,0,0,NaN,NaN,NaN,was wer very tasty


In [15]:
%%time
df1["clean_text"]=df1['Text'].apply(lambda text:tokenize(str(text)))
df1.head()

CPU times: user 14min 38s, sys: 1min 20s, total: 15min 59s
Wall time: 17min 32s


,Venue Index,Name,Date,Rating,Rating Sent,Text,Aspect,Accessibility,Term1,Term2,Term3,clean_text
Index,,,,,,,,,,,,
18594,10,coffeeshop la grotte,2019,5 stars,positive,nd year coming here and can t say enough good...,0,0,NaN,NaN,NaN,nd year good things place great staff food won...
373579,451,pata negra,2019,5 stars,positive,excellent,0,0,NaN,NaN,NaN,excellent
128693,112,bagels & beans ijdock,2021,5 stars,positive,location very nice and welcoming with a spl...,0,0,NaN,NaN,NaN,location nice splendid view large water channe...
120681,17,plum,2020,5 stars,positive,went to this place on saturday i had the gril...,0,0,NaN,NaN,NaN,place saturday cheese minute drive drive worth...
162003,504,caf‚àö¬© thijssen,2021,5 stars,positive,was wer very tasty,0,0,NaN,NaN,NaN,wer tasty


In [29]:
# Take subset of reviews discussing accessibility
df2 = df1[df1["Accessibility"] == 1]

## Creating objects required for model training

In [30]:
corpus=df2.clean_text.tolist()
vocab=list(set(word_tokenize(" ".join(df2.clean_text))))
vectorizer = CountVectorizer(ngram_range=(1,1),vocabulary=vocab)
X = vectorizer.fit_transform(corpus)
word2id=vectorizer.vocabulary_

## Defining priors

In [39]:
general_acc_words=["disability","disabled","handicap","handicapped","wheelchair","stroller",
             "mobility","reduced mobility"]

entrance_words=["ramp","entrance","wide","narrow","door"]

bathroom_words=["bathroom","toilet","restroom","handrail"]

space_words=["spacious","space","cramped","small","large","seating","room"]

stairs_words=["stair","stairs","steps","step","elevator","staircase","narrow","tricky","steep","handrail"]

## Removing prior words that are not part of the vocabulary

In [32]:
general_acc_words = [x for x in general_acc_words if x in list(word2id.keys())]
entrance_words = [x for x in entrance_words if x in list(word2id.keys())]
bathroom_words = [x for x in bathroom_words if x in list(word2id.keys())]
space_words = [x for x in space_words if x in list(word2id.keys())]
stairs_words = [x for x in stairs_words if x in list(word2id.keys())]

## Creating list of word lists as needed

In [33]:
seed_topic_list = [
    general_acc_words,
    entrance_words,
    bathroom_words,
    space_words,
    stairs_words
]

## Defining model

In [34]:
model = glda.GuidedLDA(n_topics=5, n_iter=2000, random_state=7, refresh=20,alpha=0.01,eta=0.01)

## Setting priors

In [35]:
seed_topics = {}
for t_id, st in enumerate(seed_topic_list):
    for word in st:
        seed_topics[word2id[word]] = t_id

## Training the model

In [41]:
%%time
model.fit(X, seed_topics=seed_topics, seed_confidence=0.15)

INFO:guidedlda:n_documents: 1720
INFO:guidedlda:vocab_size: 3085
INFO:guidedlda:n_words: 20416
INFO:guidedlda:n_topics: 5
INFO:guidedlda:n_iter: 2000
INFO:guidedlda:<0> log likelihood: -211686
INFO:guidedlda:<20> log likelihood: -153305
INFO:guidedlda:<40> log likelihood: -151257
INFO:guidedlda:<60> log likelihood: -150799
INFO:guidedlda:<80> log likelihood: -150115
INFO:guidedlda:<100> log likelihood: -149746
INFO:guidedlda:<120> log likelihood: -149317
INFO:guidedlda:<140> log likelihood: -149204
INFO:guidedlda:<160> log likelihood: -148742
INFO:guidedlda:<180> log likelihood: -148614
INFO:guidedlda:<200> log likelihood: -148295
INFO:guidedlda:<220> log likelihood: -148233
INFO:guidedlda:<240> log likelihood: -148100
INFO:guidedlda:<260> log likelihood: -148001
INFO:guidedlda:<280> log likelihood: -147951
INFO:guidedlda:<300> log likelihood: -148191
INFO:guidedlda:<320> log likelihood: -147736
INFO:guidedlda:<340> log likelihood: -147838
INFO:guidedlda:<360> log likelihood: -147865
I

CPU times: user 9.17 s, sys: 236 ms, total: 9.41 s
Wall time: 11.4 s


In [42]:
# Seeing the model output topics, and top 10 words per topic
n_top_words = 10
topic_word = model.topic_word_
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: food good spacious cramped service bit nice place staff great
Topic 1: spacious nice good food place great service staff terrace coffee
Topic 2: steep stairs food small place cramped delicious nice restaurant staff
Topic 3: spacious nice good place food staff terrace great service delicious
Topic 4: spacious good parking nice wheelchair amsterdam great garage accessible place


## Tagging the topics to create id - topic file

In [ ]:
topic_num_name = {"Topic 0":"general_acc_words",
                  "Topic 1":"entrance_words",
                  "Topic 2":"bathroom_words",
                  "Topic 3":"space_words",
                  "Topic 4":"stairs_words"}   

In [ ]:
def get_doc_topics(model_glda,X,num_topics,dataframe,col_name):
    """
    A function which creates dataframe with documents, their dominant topic, along with their probabilities
    
    Parameters
    -------------
    model_glda - Guided LDA trained model
    X - Document term frequency table
    num_topics - Number of topics the model was trained for
    dataframe - Dataframe consisting of cleaned text column
    col_name - Column name in dataframe holding cleaned text
    
    Returns
    -------------
    A dataframe with document number, topic, probability of topic
    """
    df_doc_top = pd.DataFrame()
    final_list = []
    for index in range(len(dataframe[col_name])):
        word_id_dict = dict((x,y) for x,y in zip([x for x in range(num_topics)],np.round(model.transform(X[index])*100,1).tolist()[0]))
        word_score_list = []
        for index in range(num_topics):
            try:
                value = word_id_dict[index]
            except:
                value = 0
            word_score_list.append(value)
        final_list.append(word_score_list)

    df_doc_top = pd.DataFrame(final_list)
    df_doc_top.columns = ['Topic ' + str(i) for i in range(num_topics)]
    df_doc_top.index = ['Document ' + str(i) for i in range(len(dataframe[col_name]))]

    df_doc_top["Dominant_Topic"] = df_doc_top.idxmax(axis=1).tolist()
    df_doc_top["Topic_Probability"] = df_doc_top.max(axis=1).tolist()
    document_df = df_doc_top.reset_index().rename(columns={"index":"Document"})[["Document","Dominant_Topic","Topic_Probability"]]

    return document_df

In [ ]:
document_df=get_doc_topics(model,X,5,df2,"clean_text")

In [ ]:
submission=pd.concat([df2.Id,document_df.Dominant_Topic],axis=1)

In [ ]:
submission.Dominant_Topic=submission.Dominant_Topic.replace(topic_num_name)

In [ ]:
submission=submission.set_index("Id").rename(columns={"Dominant_Topic":"topic"})

In [ ]:
submission